In [83]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import numpy as np
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer
import nltk
# from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [84]:
# df = pd.read_csv('/content/alice_in_wonderland.txt')
# mytext = df['question1'].tolist()
# mytext[:5]

text = ''

with open('/content/alice_in_wonderland.txt', 'r', encoding='utf-8') as f:
    text = f.readlines()

text[:10]


["TITLE: Alice's Adventures in Wonderland\n",
 'AUTHOR: Lewis Carroll\n',
 '\n',
 '\n',
 '= CHAPTER I = \n',
 '=( Down the Rabbit-Hole )=\n',
 '\n',
 '  Alice was beginning to get very tired of sitting by her sister\n',
 'on the bank, and of having nothing to do:  once or twice she had\n',
 'peeped into the book her sister was reading, but it had no\n']

In [85]:
max_seq_len = 512
vocab_size = 30000
embed_dim = 512
num_layers = 6
num_heads = 8
hidden_dim = 1024
batch_size = 16
epochs = 10
learning_rate = 1e-4
warmup_steps = 1000

In [86]:
class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len=max_seq_len):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer.encode(text, max_length=self.max_len, padding='max_length', truncation=True)
        return torch.tensor(encoding)

In [87]:
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_layers, num_heads, hidden_dim, max_seq_len):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.positional_encoding = self.get_positional_encoding(max_seq_len, embed_dim)

        encoder_layers = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dim_feedforward=hidden_dim)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)

        self.fc = nn.Linear(embed_dim, vocab_size)
        self.softmax = nn.LogSoftmax(dim=-1)

    def forward(self, x):
        device = x.device
        positional_encoding = self.positional_encoding.to(device)
        # x = self.embedding(x) + self.positional_encoding[:x.size(1), :]
        x = self.embedding(x) + positional_encoding[:, :x.size(1), :]
        x = self.transformer_encoder(x)
        x = self.fc(x)
        return self.softmax(x)

    def get_positional_encoding(self, max_seq_len, embed_dim):
        pe = torch.zeros(max_seq_len, embed_dim)
        position = torch.arange(0, max_seq_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * (-np.log(10000.0) / embed_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        return pe.unsqueeze(0)

In [88]:
texts = text
# tokenizer = word_tokenize
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
train_texts, val_texts = train_test_split(texts, test_size=0.2)

train_dataset = TextDataset(train_texts, tokenizer)
val_dataset = TextDataset(val_texts, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

model = TransformerModel(vocab_size=vocab_size, embed_dim=embed_dim, num_layers=num_layers,
                         num_heads=num_heads, hidden_dim=hidden_dim, max_seq_len=max_seq_len)
optimizer = AdamW(model.parameters(), lr=learning_rate)

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=epochs*len(train_loader))
criterion = nn.CrossEntropyLoss()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW in

In [89]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch.long().to('cuda')
            outputs = model(input_ids)

            outputs = outputs.view(-1, vocab_size)
            input_ids = input_ids.view(-1)
            loss = criterion(outputs, input_ids)
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            scheduler.step()
            total_loss += loss.item()

        print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader)}')

In [90]:
def evaluate_model(model, val_loader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids =batch.long().to('cuda')
            outputs = model(input_ids)

            outputs =outputs.view(-1, vocab_size)
            input_ids = input_ids.view(-1)
            loss = criterion(outputs, input_ids)
            total_loss +=loss.item()
    return total_loss/len(val_loader)

In [91]:
# model.to('cpu')
# currdevice = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cuda'
model.to('cuda')
train_model(model,train_loader, val_loader, criterion, optimizer,scheduler, epochs)
val_loss = evaluate_model(model, val_loader,criterion)
print(f'Validation Loss:{val_loss}')

Epoch 1/10, Loss: 3.777030020621088
Epoch 2/10, Loss: 0.1688496134347386
Epoch 3/10, Loss: 0.09224488430966933
Epoch 4/10, Loss: 0.05929569707562526
Epoch 5/10, Loss: 0.04025054987933901
Epoch 6/10, Loss: 0.028903769831069643
Epoch 7/10, Loss: 0.021493608540751868
Epoch 8/10, Loss: 0.017570449577437506
Epoch 9/10, Loss: 0.015315832109707925
Epoch 10/10, Loss: 0.014215607268528805
Validation Loss:0.018477487978008057


In [3]:
import torch
torch.save(model, 'model_full.pth')

NameError: name 'model' is not defined

In [4]:
model = torch.load('model_full.pth')
device = 'cpu'
model.to(device)

C:\Users\user\AppData\Local\Temp\ipykernel_4076\207257105.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('model_full.pth')


AttributeError: Can't get attribute 'TransformerModel' on <module '__main__'>

In [99]:
device = 'cpu'

In [100]:
def evaluate_model(model, val_loader, criterion, device):
    model.eval()  # Set the model to evaluation mode
    val_loss = 0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():  # Disable gradient calculation for evaluation
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            outputs = outputs.view(-1, vocab_size)
            labels = labels.view(-1)

            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Calculate accuracy (if classification)
            _, predicted = torch.max(outputs, dim=1)
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)

    average_loss = val_loss / len(val_loader)
    accuracy = (correct_predictions / total_samples) * 100
    print(f'Validation Loss: {average_loss:.4f}, Accuracy: {accuracy:.2f}%')


In [111]:
# def predict(model, input_texts, tokenizer, device):
#     model.eval()
#     inputs = tokenizer(input_texts, return_tensors='pt', padding=True, truncation=True, max_length=128).to(device)
#     with torch.no_grad():
#         outputs = model(**inputs)
#         # outputs = model(inputs['input_ids'], inputs['attention_mask'])
#     return outputs

def predict(model, input_texts, tokenizer, device):
    model.eval()

    # Tokenize and encode the input text
    encoded_inputs = tokenizer(input_texts, return_tensors='pt', padding=True, truncation=True, max_length=128)

    # Move tensors to the appropriate device
    input_ids = encoded_inputs['input_ids'].to(device)

    # Run inference
    with torch.no_grad():
        outputs = model(input_ids)  # Call the model with a single argument
    return outputs



In [161]:
output_tensor = predict(model,"fdsff dfsdfi s j ffsd sdfdsf sfdgdfgg sdffwefd f", tokenizer, 'cuda')
output_tensor

tensor([[[-14.0077, -18.0362, -17.8779,  ..., -17.0628, -18.8870, -18.0591],
         [-10.1765, -10.6642, -10.2550,  ...,  -9.5856, -10.9056, -10.3538],
         [ -6.1384, -10.6973,  -9.7426,  ..., -11.2020, -11.7542, -11.1571],
         ...,
         [ -8.1554, -10.8178, -10.9072,  ..., -10.6442, -10.8703, -10.8024],
         [-10.1148, -10.6818, -10.4188,  ...,  -9.7317, -10.6712, -10.4630],
         [-13.6406, -14.9259, -15.9283,  ..., -14.6627, -15.4674, -15.3328]]],
       device='cuda:0')

In [162]:
output_tensor.shape

torch.Size([1, 28, 30000])

In [163]:
def process_language_model_output(output_tensor, tokenizer):
    # Assuming output_tensor contains token IDs
    predicted_token_ids = torch.argmax(output_tensor, dim=-1)
    decoded_texts = tokenizer.batch_decode(predicted_token_ids, skip_special_tokens=True)
    return decoded_texts

In [164]:
process_language_model_output(output_tensor, tokenizer)

['( she great d s very syp alice all what she whatd " very only, arm whatd -']